1. Cargar y explorar un archivo JSON

In [1]:
import pandas as pd
import json
path = r'C:\Users\Diego\Documents\GitHub\mitic-data-science-team-1-septiembre-2024\clases\ds-fundamentals\data\ejercicios_integracion\competencia_kaggle.json'

# 1. Cargar el archivo JSON
with open(path, 'r') as file:
    data = json.load(file)

# Convertir los datos JSON a un DataFrame
df = pd.json_normalize(data, 'puntuaciones', ['id_participante', 'nombre', 'intentos', 'tiempo_total_horas'], record_prefix='puntuacion_')
df.head(6)
df.dtypes

puntuacion_0          float64
id_participante        object
nombre                 object
intentos               object
tiempo_total_horas     object
dtype: object

In [2]:
#Convertir intentos y tiempo total a valores numericos
df['intentos'] = pd.to_numeric(df['intentos'], errors='coerce')
df['tiempo_total_horas'] = pd.to_numeric(df['tiempo_total_horas'], errors='coerce')

2. Identificar y manejar datos nulos.

In [3]:

# Cambiar NaN en Puntuacion con el promedio de puntuaciones por participante
df['puntuacion_0'] = df.groupby('id_participante')['puntuacion_0'].transform(lambda x: x.fillna(x.mean()))

# Cambiar por el NaN por el promedio de horas total.
df.fillna({'tiempo_total_horas': df['tiempo_total_horas'].mean()})

df.head()

,puntuacion_0,id_participante,nombre,intentos,tiempo_total_horas
0,80.00,1,Participante_A,5,12.0
1,85.00,1,Participante_A,5,12.0
2,90.00,1,Participante_A,5,12.0
3,88.00,1,Participante_A,5,12.0
4,85.75,1,Participante_A,5,12.0


3. Calcular estadísticas descriptivas (promedios, máximos, mínimos, etc.) para el número de intentos, el tiempo invertido y las puntuaciones.

In [4]:
print("Estadísticas de intentos:\n", df['intentos'].describe())
print("Estadísticas de tiempo total:\n", df['tiempo_total_horas'].describe())
print("Estadísticas de puntuaciones:\n", df['puntuacion_0'].describe())


Estadísticas de intentos:
 count    20.000000
mean      4.500000
std       1.357242
min       2.000000
25%       3.750000
50%       5.000000
75%       6.000000
max       6.000000
Name: intentos, dtype: float64
Estadísticas de tiempo total:
 count    17.000000
mean     11.411765
std       2.873561
min       5.000000
25%      10.000000
50%      12.000000
75%      14.000000
max      14.000000
Name: tiempo_total_horas, dtype: float64
Estadísticas de puntuaciones:
 count    20.000000
mean     80.837500
std       9.581807
min      60.000000
25%      77.250000
50%      81.625000
75%      86.312500
max      95.000000
Name: puntuacion_0, dtype: float64


4. Aplicar condicionales y operadores matemáticos para evaluar el rendimiento de los participantes (e.g., calcular el promedio de puntuaciones por intento).

In [5]:

df['promedio_por_intento'] = df.groupby('id_participante')['puntuacion_0'].transform('mean')

# 6. Resumen del rendimiento por participante
rendimiento_participantes = df.groupby(['id_participante', 'nombre']).agg(
    intentos=('intentos', 'first'),
    tiempo_total_horas=('tiempo_total_horas', 'first'),
    promedio_puntuacion=('promedio_por_intento', 'mean')
).reset_index()



5. Guardar el conjunto de datos limpio en un nuevo archivo JSON.

In [6]:
# 7. Guardar el DataFrame limpio y resumido en un nuevo archivo JSON
output_path = r"C:\Users\Diego\Documents\GitHub\mitic-data-science-team-1-septiembre-2024\clases\ds-fundamentals\data\ejercicios_integracion\competencia_kaggle_limpio.json"
rendimiento_participantes.to_json(output_path, orient='records', lines=True)


print(f"\nArchivo JSON con el rendimiento de los participantes guardado en: {output_path}")


Archivo JSON con el rendimiento de los participantes guardado en: C:\Users\Diego\Documents\GitHub\mitic-data-science-team-1-septiembre-2024\clases\ds-fundamentals\data\ejercicios_integracion\competencia_kaggle_limpio.json
